# Visualize ERP by trial 2D matrix (imagesc)

In [ ]:
#Importing files and modules

import numpy as np
import matplotlib as mpl
import matplotlib.colors as colors
from os import path, listdir
from brainpipe.system import study
import matplotlib.pyplot as plt
%matplotlib notebook

from brainpipe.visual import *
from mne.baseline import rescale
from mne.filter import filter_data

In [ ]:
# Variables 
baseline = [640 , 768] #-250ms à 0ms
data_to_use = [768, 1536]
vlines = [0]
sf = 512.
elec = 0

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_learningOK/')

subjects = ['CHAF','VACJ','SEMC','PIRJ','LEFC','MICP',]

conds = ['all']

n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
}

for su in subjects:
    for elec in range(0, n_elec[su],1):
        info_elec = np.load(path.join(path_data, su+'_E1E2_concat_learning_0_bipo.npz'))
        channel, label = info_elec['channel'], info_elec['label']
        file0 = np.load(path.join(path_data, su+'_E1E2_concat_learning_0_bipo.npz'))['x'] 
        file1 = np.load(path.join(path_data, su+'_E1E2_concat_learning_1_bipo.npz'))['x']
        file2 = np.load(path.join(path_data, su+'_E1E2_concat_learning_2_bipo.npz'))['x']
        print ('processing --> ', file0.shape)
        data_all = np.concatenate((file0, file1, file2), axis = 2)
        data_elec = data_all[elec,:,:]
        print ('data elec size : ', data_elec.shape)

        #Filter data for one elec (all trials):
        data_elec = np.array(data_elec, dtype='float64')
        data_to_filter = np.swapaxes(data_elec, 0, 1)
        filtered_data = filter_data(data_to_filter, sfreq=512, l_freq=None, h_freq=10., method='fir', phase='zero-double')
        filtered_data = np.swapaxes(filtered_data, 0, 1)
        print ('Size of filtered data:', filtered_data.shape,) # ntimes * trials

        #Select data to plot
        filtered_data_elec = filtered_data[range(baseline[0], data_to_use[1])]
        filtered_data_elec = filtered_data_elec.swapaxes(0,1)
        print ('filtered selected data : ', filtered_data_elec.shape)

        #Normalize the non-averaged data (all trials)
        times = np.arange(filtered_data_elec.shape[1])
        print ('time points : ', times.shape)
        norm_filtered_data = rescale(filtered_data_elec, times=times, baseline=baseline, mode='mean')
        print ('Size norm & filtered data : ', norm_filtered_data.shape,)

        #Prepare data to plot
        ntrials = norm_filtered_data.shape[0]
        hlines = [ntrials/3, ntrials*2/3]
        #times_plot = 1000 * np.arange((baseline[0] - baseline[1]), data_elec_to_plot.shape[0]-baseline[1] + baseline[0],) / sf
        times_plot = 1000 * np.arange((baseline[0] - baseline[1]), norm_filtered_data.shape[1]-baseline[1] + baseline[0],)/sf

        #Plot data
        plt.imshow(norm_filtered_data, aspect=30, norm=colors.Normalize(vmin=-50,vmax=50),
                   interpolation='none', origin ='lower', extent = [times_plot[0], times_plot[-1], 0, ntrials] ) 
        addLines(plt.gca(), vLines=vlines, vColor=['firebrick']*2, vWidth=[1.5]*2, hLines=hlines, hColor=['#000000']*2, hWidth=[1.5]*2)
        plt.axis('tight')
        ax = plt.gca()
        ax.set_xlabel('Times (ms)', fontsize=13)
        ax.set_ylabel('Trials (#)', fontsize=13)
        ax.set_title(su+' : ERPs by trial / Elec : '+str(elec)+' '+label[elec]+' '+channel[elec], fontsize=16,fontweight=4  )       
        plt.colorbar()
        
# =========================SAVE PLOTS of ERPs=================================================
        rep = path.join(st.path, 'feature/ERP_by_trial_Filtered_rescale_Encoding_all_bipo_250ms_mean_thr40_art400_30_250/',su)
        fname = (rep + '_E1E2_ERP_2D_matrix_by_trial_concat_all_bipo_' + channel [elec] +'_'+str(elec)+'_'+label[elec]+'.png')
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.clf()
        plt.close()
del x, channel, n_elec, ntrials, label


In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250/')
elec = 23 

filename = 'VACJ_E1E2_concat_allfilter1_bipo.npz'
print ('processing --> ', filename)
data_all = np.load(path.join(path_data, filename))
data, channel, label = data_all['x'], data_all['channel'], data_all['label']
data_elec = data[elec,:,:]
print ('data elec size : ', data_elec.shape)

#Filter data for one elec (all trials):
data_elec = np.array(data_elec, dtype='float64')
data_to_filter = np.swapaxes(data_elec, 0, 1)
filtered_data = filter_data(data_to_filter, sfreq=512, l_freq=None, h_freq=10., method='fir', phase='zero-double')
filtered_data = np.swapaxes(filtered_data, 0, 1)
print ('Size of filtered data:', filtered_data.shape,) # ntimes * trials

#Select data to plot
filtered_data_elec = filtered_data[range(baseline[0], data_to_use[1])]
filtered_data_elec = filtered_data_elec.swapaxes(0,1)
print ('filtered selected data : ', filtered_data_elec.shape)

#Normalize the non-averaged data (all trials)
times = np.arange(filtered_data_elec.shape[1])
print ('time points : ', times.shape)
norm_filtered_data = rescale(filtered_data_elec, times=times, baseline=baseline, mode='mean')
print ('Size norm & filtered data : ', norm_filtered_data.shape,)

#Prepare data to plot
ntrials = norm_filtered_data.shape[0]
hlines = [ntrials/3, ntrials*2/3]
#times_plot = 1000 * np.arange((baseline[0] - baseline[1]), data_elec_to_plot.shape[0]-baseline[1] + baseline[0],) / sf
times_plot = 1000 * np.arange((baseline[0] - baseline[1]), norm_filtered_data.shape[1]-baseline[1] + baseline[0],)/sf
vmin = norm_filtered_data.min()
vmax = norm_filtered_data.max()
print ('vmin', vmin, 'vmax', vmax)
#Plot data
#plt.figure(figsize=(10,20))
#plt.subplot(2, 1, 1)
plt.imshow(norm_filtered_data, aspect=30, norm=colors.Normalize(vmin=-50,vmax=50), 
           interpolation='none', origin ='lower', extent = [times_plot[0], times_plot[-1], 0, ntrials] ) 
addLines(plt.gca(), vLines=vlines, vColor=['firebrick']*2, vWidth=[1.5]*2, hLines=hlines, hColor=['#000000']*2, hWidth=[1.5]*2)
plt.axis('tight')
ax = plt.gca()
ax.set_xlabel('Times (ms)', fontsize=13)
ax.set_ylabel('Trials (#)', fontsize=13)
ax.set_title('ERPs by trial / Elec : '+str(elec), fontsize=16,fontweight=4  ) 
#plt.subplot(2,1,2)
#plt.imshow(data_elec_to_plot, aspect='auto', interpolation='none', origin ='lower',) 
#addLines(plt.gca(), vLines=vlines, vColor=['firebrick']*2, vWidth=[1.5]*2, hLines=hlines, hColor=['#000000']*2, hWidth=[1.5]*2)
#plt.axis('off')

plt.show()
plt.colorbar()


In [ ]:
cmap = mpl.cm.cool
norm = mpl.colors.Normalize(vmin=5, vmax=10)
plt.show()